In [1]:
import os
import pandas as pd
import numpy as np
import scipy
import datetime
from sklearn.model_selection import train_test_split
import gc, time
#os.chdir('C:/Users/Paperspace/Desktop/Kaggle-Competitions/FraudDetection')
os.chdir('C:/Users/dhruv/Desktop/Kaggle/TalkingData/data')

In [2]:
dtypes = {
    'ip'            : 'uint32',
    'app'           : 'uint16',
    'device'        : 'uint16',
    'os'            : 'uint16',
    'channel'       : 'uint16',
    'is_attributed' : 'uint8',
    'click_id'      : 'uint32'
}

In [3]:
def handleClicktime(df):
    df['click_hour']= (pd.to_datetime(df['click_time']).dt.round('H')).dt.hour
    df['click_hour'] = df['click_hour'].astype('uint16')
    df = df.drop(['click_time'], axis=1)   
    return df

In [11]:
st_time = time.time()
#train = pd.read_csv('train_sample.csv', dtype=dtypes, skiprows=range(1,133333333), nrows=33333333)
train = pd.read_csv('train_sample.csv', dtype=dtypes, nrows=10000)
print ('Time to load train [{}] sec'.format(time.time()-st_time))

train_record_index = train.shape[0]

Time to load train [0.025068283081054688] sec


In [12]:
st_time = time.time()
test = pd.read_csv('test.csv', nrows=10000)
print ('Time to load test [{}] sec'.format(time.time()-st_time))

Time to load test [0.021057844161987305] sec


In [13]:
# Handle click time
st_time = time.time()
train = handleClicktime(train)
test = handleClicktime(test)
print ('Time to handle click time [{}] sec'.format(time.time()-st_time))
gc.collect()

Time to handle click time [0.17690253257751465] sec


298

In [15]:
#df for submit
df_submit = pd.DataFrame()
df_submit['click_id'] = test['click_id']

In [16]:
# Get y
Learning_Y = train['is_attributed']

In [17]:
#drop zone
test = test.drop(['click_id'], axis=1)
train = train.drop(['is_attributed', 'attributed_time'], axis=1)
gc.collect()

206

In [18]:
df_merge = pd.concat([train, test])
del train, test
gc.collect()

0

In [19]:
# Count ip for both train and test df 
start_time = time.time()
df_ip_count = df_merge['ip'].value_counts().reset_index(name='ip_count')
df_ip_count.columns = ['ip', 'ip_count']
print('Load df_ip_count with [{}] seconds'.format(time.time() - start_time))

Load df_ip_count with [0.021090030670166016] seconds


In [20]:
df_merge = df_merge.merge(df_ip_count, on='ip', how='left', sort=False)
df_merge['ip_count'] = df_merge['ip_count'].astype('uint16')

In [21]:
df_merge = df_merge.drop(['ip'], axis=1)
del df_ip_count
gc.collect()

42

In [22]:
train = df_merge[:train_record_index]
test = df_merge[train_record_index:]

del df_merge
gc.collect()

0

# MLP classifier

In [24]:
from sklearn.neural_network import MLPClassifier

In [30]:
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(5, 2), batch_size = 100000, random_state=42)
clf.fit(train, Learning_Y)

C:\Users\dhruv\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:357: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn("Got `batch_size` less than 1 or larger than "
C:\Users\dhruv\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


MLPClassifier(activation='relu', alpha=1e-05, batch_size=100000, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=42, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [31]:
# predict
predictions = clf.predict_proba(test)

In [32]:
df_submit['is_attributed'] = predictions[:,1]
df_submit.describe()
df_submit.to_csv('submission.csv', index = False)